In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from sklearn.multiclass import OneVsRestClassifier

In [21]:
traindf = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
testdf = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")

In [22]:
traindf

In [24]:
traindf[traindf['toxic']==1]

In [25]:
import re
import string

In [26]:
def  clean_text(text):
    text =  text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    
    return text

In [27]:
traindf['comment_text'] = traindf['comment_text'].apply(lambda x:clean_text(x))

In [28]:
testdf['comment_text'] = testdf['comment_text'].apply(lambda x:clean_text(x))

In [29]:
traindf.info()

In [30]:
traindf.isna().sum()

In [31]:
X =  traindf.comment_text
y =  traindf.drop(['id','comment_text'],axis = 1)

In [32]:
X_train,X_valid, y_train,y_valid= train_test_split(X,y,test_size=0.2, random_state=1)

In [36]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',     
    analyzer='word',            
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words='english',
    sublinear_tf=True)

word_vectorizer.fit(X_train)    
train_word_features = word_vectorizer.transform(X_train)

In [37]:
X_train_transformed = word_vectorizer.transform(X_train)
X_valid_transformed = word_vectorizer.transform(X_valid)

In [ ]:
log_reg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=45)

classifier = OneVsRestClassifier(log_reg)
classifier.fit(X_train_transformed, y_train)


y_train_pred_proba = classifier.predict_proba(X_train_transformed)
y_valid_pred_proba = classifier.predict_proba(X_valid_transformed)


roc_auc_score_train = roc_auc_score(y_train, y_train_pred_proba,average='weighted')
roc_auc_score_test = roc_auc_score(y_valid, y_valid_pred_proba,average='weighted')

print("ROC AUC Score Train:", roc_auc_score_train)
print("ROC AUC Score Test:", roc_auc_score_test)

In [ ]:
def make_test_predictions(df,classifier):
    df.comment_text = df.comment_text.apply(clean_text)
    X_test = df.comment_text
    X_test_transformed = word_vectorizer.transform(X_test)
    y_test_pred = classifier.predict_proba(X_test_transformed)
    result =  sum(y_test_pred[0])
    if result >=1 :
       return("Toxic Comment")
    else :
       return ("NonToxic Comment")

In [11]:
from sklearn.pipeline import Pipeline

In [12]:
pipe = Pipeline([
    ('word_vectorizer',word_vectorizer),
    ('clf',OneVsRestClassifier(LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=45)))
])

In [13]:
pipe.fit(X_train,y_train)

In [15]:
y_pred = pipe.predict(X_valid)
accuracy_score(y_valid,y_pred)

In [35]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [43]:
y_pred1 = pipe.predict(["You are ugly"])

In [44]:
y_pred1